# Notes
- Different species
- Gap phase replacement
- Spatial field

# To-do
- Inform initial spatial and size distributions
- Inform reproductive mechanism (range, probability, etc.)
- Inform mortality mechanism
- Make the viewer more interactive (zoom and move around, play, reverse, speed up, slow down)
- ~~Write your own quadtree class and add a method for removing objects from it to speed up collision detection~~
- ~~Implement plant reproduction (either as a method in the Plant class or in the Simulation class)~~
- ~~Fix new plant spawns showing up for one frame and then getting killed~~
- ~~Add find collisions to Simulation class~~
- ~~Move plants list and states list to Simulation class~~
- ~~Fix Plant .kill() method~~
- ~~replace .update_color() method to set_color() in Plant class~~
- ~~add simulation.step() method to Simulation class~~

# Intersection probability (unfinished)
### Probability of a Point Inside Circles in a Square
**Problem Setup**
- Square with side length $S$
- $N$ circles with radii uniformly distributed between $r_{min}$ and $r_{max}$
- Find the probability that a new random point $p$ is inside at least one circle

**1. Expected Area of a Single Circle**

$$E[A] = \pi E[r^2] = \pi(r_{min}^2 + r_{max}^2 + r_{min}r_{max}) / 3$$

**2. Total Expected Area Covered by Circles**

$$E[A_\mathrm{total}] = N \cdot E[A]$$

**3. Probability Calculation**

The probability of the point being inside a circle is equal to the fraction of the square's area that is covered by circles, on average:

$$P(p \text{ inside at least one circle}) = \frac{E[A_\mathrm{total}]}{S^2} = \frac{N \cdot \pi \cdot (r_{min}^2 + r_{max}^2 + r_{min}r_{max})}{3S^2}$$

**4. Edge Cases**

- If $E[A_\mathrm{total}] > S^2$, cap the probability at 1
- The formula assumes circles can overlap and extend beyond the square's boundaries

## Notes

This approach uses the concept of expected value and assumes that the circles are placed randomly and independently. It's an average case analysis - the actual probability for a specific configuration of circles might differ.

For very large $N$ or when circles are large relative to the square, this simple formula might overestimate the probability due to increased overlap. In such cases, more sophisticated models from continuum percolation theory might be needed.

## Key Reference

Hall, P. (1988). Introduction to the Theory of Coverage Processes. John Wiley & Sons.

This book discusses various aspects of coverage problems, including the one we've just solved.



*WRITTEN BY CLAUDE AI*

**Define the problem parameters.**
- Square side length: $S$
- Number of circles: $N$
- Distribution of circle radii: Let's assume a uniform distribution between $r_\mathrm{min}$ and $r_\mathrm{max}$ for simplicity


**Expected area of a single circle.**
$$E[A] = \pi \cdot E[r^2]$$
For a uniform distribution: $E[r^2] = (r_\mathrm{min}^2 + r_\mathrm{max}^2 + r_\mathrm{min}r_\mathrm{max}) / 3$


**Total expected area covered by all circles.**
$$E[A_\mathrm{total}] = N \cdot E[A]$$

**Probability of a random point being covered by at least one circle**
$$P(\mathrm{covered}) = 1 - \exp\left(-\frac{E[A_\mathrm{total}]}{S^2}\right)$$

This step uses the void (avoidance) probability from the Boolean model in stochastic geometry. The $\exp(-\mathrm{density})$ term gives the probability of a point not being covered by any circle (which is exactly the void probability).
The probability of overlap is related to this coverage probability. If the circles are independently placed, we can approximate the probability of overlap as:

$$P(\mathrm{overlap}) ≈ 1 - (1 - P(\mathrm{covered}))^N$$

This is because for no overlap to occur, each circle must fall entirely in the uncovered region of all previous circles.
For more accurate results, especially with larger circles or higher densities, you'd need to account for edge effects and higher-order interactions. This often requires numerical simulations or more advanced analytical techniques.


Let's summarize into one equation

$$P(\mathrm{overlap}) = 1 - \exp\left( -\frac{E[A_\mathrm{total}]}{S^2} \right)^N = 1 - \exp\left(- \frac{N\pi}{3} \cdot \frac{r_\mathrm{min}^2 + r_\mathrm{max}^2 + r_\mathrm{min}r_\mathrm{max}}{S^2} \right)^N$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def diff(EA, A_box, Q_land, Π_species, Π_death):
    def dPdt(P, t):
        density = P * EA / A_box

        Π_point = (1 - density)
        Π_overlap = (1 - np.exp(-density))

        Π_reproduction = Π_point * (density + Q_land) * Π_species

        Π_competition = Π_overlap/2

        # print(density, Π_reproduction, Π_competition, Π_death, Π_reproduction - Π_competition - Π_death)
        
        return P * (Π_reproduction - Π_competition - Π_death)
    return dPdt

n_iter = 1000
P = num_plants = 1_000
S = 1_000 # m

A_box = S**2

growth_rate = 0.1
r_min = 0.01
r_max = 30
Π_death = growth_rate/(r_max - r_min)

dPdt = diff(EA=EA, A_box=A_box, Q_land=-0.1, Π_species=0.2, Π_death=Π_death)

P = P0 = N
Ps = [P0]
for t in range(n_iter):
    P += dPdt(P, t)
    if P < 0:
        P = 0

    Ps.append(P)

plt.plot(Ps)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
rs = np.array([np.random.rand() for i in range(1000)])
r_mean = np.mean(rs)
r2_mean = np.mean(rs**2)

print(f'{r_mean = :.3f}, {r2_mean = :.3f}, {r_mean**2 = :.3f}, {np.sqrt(r2_mean) = :.3f}')

In [ ]:
from scipy.interpolate import RegularGridInterpolator
import numpy as np
import matplotlib.pyplot as plt

half_width = 0.5
half_height = 0.5
resolution = 5
shape = (I, J) = (resolution, resolution)
print(f'{shape = }')

field = np.zeros(shape)
field[shape[0]//2 - resolution//4, shape[1]//2] = 1
xx = np.linspace(-half_width, half_width, resolution)
yy = np.linspace(-half_height, half_height, resolution)

# Create an interpolating function
interp_func = RegularGridInterpolator((xx, yy), field, method='linear')

# Define finer resolution
fine_resolution = 11
xx_fine = np.linspace(-half_width, half_width, fine_resolution)
yy_fine = np.linspace(-half_height, half_height, fine_resolution)

# Interpolate the field
X_fine, Y_fine = np.meshgrid(xx_fine, yy_fine)
points_fine = np.array([X_fine.flatten(), Y_fine.flatten()]).T
print(points_fine.shape)
field_fine = interp_func(points_fine).reshape(fine_resolution, fine_resolution).T

print()
print(f'field = \n{np.round(field, 2)}')
print()
print(f'field_fine = \n{np.round(field_fine, 2)}')


fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(field, origin='upper', extent=(-half_width, half_width, -half_height, half_height))
ax[0].set_aspect('equal', adjustable='box')
# fig.colorbar(ax[0].collections[0], ax=ax[0])
ax[0].set_title('Initial field')

ax[1].imshow(field_fine, origin='upper', extent=(-half_width, half_width, -half_height, half_height))
ax[1].set_aspect('equal', adjustable='box')
# fig.colorbar(ax[1].collections[0], ax=ax[1])
ax[1].set_title('Interpolated field')

plt.show()

In [ ]:
import numpy as np
from numba import njit, prange
import matplotlib.pyplot as plt
from mods.fields import getPairwiseSeparations, W

@njit(parallel=True)
def getDensity(r, pos, m, hSq):
    M = r.shape[0]
    N = pos.shape[0]

    if N == 0:
        return np.zeros((M, 1))

    dx, dy = getPairwiseSeparations(r, pos)

    rho = np.zeros(M)
    for i in prange(M):
        for j in prange(N):
            rho[i] += m[j] * W(dx[i, j], dy[i, j], hSq)

    return rho.reshape((M, 1))


n = 100
res = 100
r = np.random.rand(n, 2)
xx = np.linspace(0, 1, res)
X, Y = np.meshgrid(xx, xx)
grid_points = np.array([X.ravel(), Y.ravel()]).T
m = np.random.rand(n)
hSq = 0.1**2

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.scatter(r[:, 0], r[:, 1], s=1)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal', adjustable='box')

density = getDensity(r, grid_points, m, hSq)

plt.imshow(density.reshape(res, res), origin='upper', extent=(0, 1, 0, 1))

In [ ]:
from plant import Plant

def check_pos_collision(pos, plant):
    return np.sum((pos - plant.pos) ** 2) < plant.r ** 2

plant_kwargs = {
    'r_min': 0.01,
    'r_max': 30,
    'growth_rate': 0.01,
    'dispersal_range': 100,
    'species_germination_chance': 0.025,
}

plants = [Plant(np.array([0, -0.2]), **plant_kwargs), Plant(np.array([0, 0.1]), **plant_kwargs), Plant(np.array([0.0, -0.1]), **plant_kwargs)]
this_plant = plants[-1]

fig, ax = plt.subplots()
for plant in plants:
    ec = 'r' if plant == this_plant else 'b'
    circle = plt.Circle(plant.pos, plant.r, edgecolor=ec, facecolor='none')
    ax.add_patch(circle)
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.set_aspect('equal', 'box')
plt.title('Collision detection, debugging')
plt.show()

indices = [j for j, other in enumerate(plants) if other != this_plant]
print(f'{indices = }')
plants_nearby = [plants[i] for i in indices]
print([(plant.pos, plant.r) for plant in plants])
print([(plant.pos, plant.r) for plant in plants_nearby])

collisions_nearby = np.array([check_pos_collision(this_plant.pos, plant) for plant in plants_nearby])
print(collisions_nearby)

p = None
if len(plants_nearby) == 0:
    p =  0
elif collisions_nearby.any():
    print('collision')
    p =  0
else:
    print('no collision')
    p = 1
    
print(p)

In [ ]:
import numpy as np
times = []
size = 5
skip = 20
preset_times = [1, 25]

fields = np.zeros((size, 1, 1))

def add(field, t):
    global fields, times
    if len(times) == size:
        for i, time in enumerate(times):
            if time not in preset_times:
                print(f'{time = } not in {preset_times = }')
                B = np.roll(fields[i:], -1, axis=0)
                fields = np.concatenate((fields[:i], B), axis=0)
                times.pop(i)
                break
          
    fields[len(times)] = field
    times.append(t)

for t in range(0, 200):
    if t % skip == 0 or t in preset_times:
        
        add(field=np.ones(fields.shape[-2:]) * t, t=t)
        print(f'{t = }')
        print(f'{times = }')
        print(f'{fields = }')
        

# print(fields)
# print(times)


In [ ]:
import numpy as np
ts = []
size = 4
A = np.ones((size, 2, 2))

for j in range(len(A)):
    A[j] = A[j] * j
    ts.append(j)

print(A)
print(ts)

i = 1  # could be any number

A[i] = np.ones(A.shape[-2:]) * (len(A))
B = np.roll(A[i:], (-i), axis=0)
A = np.concatenate((A[:i], B), axis=0)
ts.pop(i)
ts.append(len(A))

print()
print(A)
print(ts)

# Sampling lifetime uniformly is equivalent to sampling radii uniformly because

$$T(r) = \frac{r_{\mathrm{max}} - r}{g}, \qquad \text{where } g \text{ is the constant growth rate}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

growth_rate = 0.1
r_min = 0.00
r_max = 30

rs = np.array([np.random.uniform(r_min, r_max) for i in range(100_000)])

lifetimes = (r_max - rs)/growth_rate

fig, ax = plt.subplots(1,2 , figsize=(10, 3))
bins = np.linspace(0, r_max, 100)
ax[0].set_title('Plant radii')
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Radius')
ax[0].hist(rs, bins=bins, alpha=0.5, color='b');

bins = np.linspace(0, (r_max - r_min)/growth_rate, 100)
ax[1].set_title('Plant lifetimes (x-axis flipped)')
ax[1].set_ylabel('Frequency')
ax[1].set_xlabel('Lifetime')
ax[1].hist(lifetimes, bins=100, alpha=0.5, color='r');
ax[1].invert_xaxis()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#### FOLLOWING TABLE 5 FROM CUMMINGS ET AL. (2002) ####

names = ['<10', '10-30', '30-50', '50-70', '70-100', '100-200', '200-300']
freq_all_forests = np.array([6300, 350.7, 50.70, 12.0, 3.7, 1.5, 0.4])
freq_dense_forest = np.array([5800, 378.8, 50.98, 13.42, 5.62, 0.73, 0.73])
freq_open_forest = np.array([7500, 359.5, 51.90, 13.49, 1.75, 2.06, 0])

print(f'The total density of trees in all forests is \t {np.sum(freq_all_forests):.3f} trees/ha or \t {10_000/np.sum(freq_all_forests):.3f} m^2/tree')
print(f'The total density of trees in dense forests is \t {np.sum(freq_dense_forest):.3f} trees/ha or \t {10_000/np.sum(freq_dense_forest):.3f} m^2/tree')
print(f'The total density of trees in open forests is \t {np.sum(freq_open_forest):.3f} trees/ha or \t {10_000/np.sum(freq_open_forest):.3f} m^2/tree')


fig, ax = plt.subplots(1, 1, figsize=(6, 6))

# Define the width of the bars
bar_width = 0.1

# Calculate the positions for each set of bars
x = np.arange(len(names))
x_all_forests = x - bar_width
x_dense_forest = x
x_open_forest = x + bar_width

# Plot the bars
ax.bar(x_all_forests, freq_all_forests, width=bar_width, label='All forests', align='center')
ax.bar(x_dense_forest, freq_dense_forest, width=bar_width, label='Dense forest', align='center')
ax.bar(x_open_forest, freq_open_forest, width=bar_width, label='Open forest', align='center')

# Set the x-ticks to the middle of the groups
ax.set_xticks(x)
ax.set_xticklabels(names)

ax.set_yscale('log')
ax.set_xlabel('Tree size [cm bdh]')
ax.set_ylabel('Ferquency')
ax.legend()

fig.suptitle('Tree size distribution in open and dense forests')
fig.tight_layout()
plt.show()



# Kernel density estimation for plant size distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

#### FOLLOWING TABLE 5 FROM CUMMINGS ET AL. (2002) ####

names = ['<10', '10-30', '30-50', '50-70', '70-100', '100-200', '200-300']
means = np.array([5, 20, 40, 60, 85, 150, 250])
freqs = np.array([5800, 378.8, 50.98, 13.42, 5.62, 0.73, 0.73])

# Apply log transformation to the data
log_means = np.log(means)

# Calculate the KDE of the log-transformed data
kde = gaussian_kde(log_means, weights=freqs, bw_method='silverman')

# # Define the range of the KDE in the original space
# x = np.linspace(0.1, 10000, 10000)  # Avoid log(0) by starting from a small positive value
# log_x = np.log(x)
# y = kde(log_x) / x  # Adjust the KDE values for the log transformation

# Resample from the KDE and transform back to the original space
num_plants = 10000
log_samples = kde.resample(num_plants)[0]
samples = np.exp(log_samples)

# Plot the histogram of the samples
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.hist(samples, bins=30, density=True, alpha=0.5, color='g', label='Samples')
# ax.plot(x, y, label='KDE', color='r')
ax.set_xlim(0, samples.max())
ax.set_yscale('log')
ax.set_xlabel('Tree size [cm bdh]')
ax.set_ylabel('Density')
ax.legend()
plt.show()

# Kernel density estimation for plant density

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from scipy.spatial import KDTree
from scipy.stats import gaussian_kde as kde

class custom_kde(gaussian_kde):
    def __init__(self, dataset, bw_method=None, weights=None):
        super().__init__(dataset, bw_method=bw_method, weights=weights)

    def _compute_covariance(self):
        """Computes the covariance matrix for each Gaussian kernel using
        covariance_factor().
        """
        self.factor = self.covariance_factor()
        self._data_covariance = np.eye(self.dataset.shape[0])
        self._data_cho_cov = np.eye(self.dataset.shape[0])
        self.covariance = self._data_covariance * self.factor**2
        self.cho_cov = (self._data_cho_cov * self.factor).astype(np.float64)
        self.log_det = 2*np.log(np.diag(self.cho_cov
                                        * np.sqrt(2*np.pi))).sum()

np.random.seed(0)
m2pp = 10000
ns = [10, 100, 1000, 10000]

for i, n in enumerate(ns):
    fig, ax = plt.subplots(figsize=(4, 4))
    _m = float(np.sqrt(1/(m2pp * n)))
    print(f'{n = }, {_m = }')
    # Define the grid on which to evaluate the KDE

    positions1 = np.random.normal(loc=[-0.3, -0.3], scale=0.1, size=(n//3, 2))
    positions2 = np.random.normal(loc=[0.3, 0.3], scale=0.1, size=(n//3, 2))
    positions3 = np.random.normal(loc=[0.0, 0.0], scale=0.1, size=(n//3, 2))
    positions = np.concatenate((positions1, positions2, positions3), axis=0)
    positions = positions[(positions[:, 0] >= -0.5) & (positions[:, 0] <= 0.5) & (positions[:, 1] >= -0.5) & (positions[:, 1] <= 0.5)]
    areas = np.pi* (np.random.uniform(0.01, 30, len(positions)) * _m)**2

    bw = 100*_m
    res = 100

    x = np.linspace(-0.5, 0.5, res)
    X, Y = np.meshgrid(x, x)
    XY_vstack = np.vstack([X.ravel(), Y.ravel()])

    # Calculate the kernel density estimate
    kde = custom_kde(positions.T, bw_method=bw, weights=areas)
    Z = kde(XY_vstack).reshape(X.shape)
    im = ax.imshow(Z, origin='lower', extent=(x[0], x[-1], x[0], x[-1]), cmap='Greys')
    # ax.contour(X, Y, Z, levels=[0.1, 0.5, 1.0], colors=['r', 'g', 'b'][::-1], alpha = 0.5, linewidths=1000*_m)
    ax.scatter(positions[:, 0], positions[:, 1], s=areas*1000, c='magenta', alpha=0.3)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f'n = {n}')
    fig.colorbar(im, ax=ax)

    fig.dpi = 100
    fig.tight_layout()
    plt.show()


### Smoothed-particle hydrodynamics density estimation
Taken from https://github.com/pmocz/sph-python/blob/master/sph.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def W(x, y, h):
	"""
	Gaussian Smoothing kernel (2D)
	x     is a vector/matrix of x positions
	y     is a vector/matrix of y positions
	h     is the smoothing length
	w     is the evaluated smoothing function
	"""
	r = np.sqrt(x**2 + y**2)
	w = 1/(2 * np.pi * h**2) * np.exp(-r**2 / (2 * h**2))
	return w

def getPairwiseSeparations(ri, rj):
	"""
	Get pairwise separations between 2 sets of coordinates
	ri    is an M x 2 matrix of positions
	rj    is an N x 2 matrix of positions
	dx, dy   are M x N matrices of separations
	"""
	M = ri.shape[0]
	N = rj.shape[0]
	rix = ri[:,0].reshape((M,1))
	riy = ri[:,1].reshape((M,1))
	rjx = rj[:,0].reshape((N,1))
	rjy = rj[:,1].reshape((N,1))
	dx = rix - rjx.T
	dy = riy - rjy.T
	return dx, dy

def getDensity(r, pos, m, h):
	"""
	Get Density at sampling locations from SPH particle distribution
	r     is an M x 2 matrix of sampling locations
	pos   is an N x 2 matrix of SPH particle positions
	m     is the particle mass
	h     is the smoothing length
	rho   is M x 1 vector of densities
	"""
	M = r.shape[0]
	dx, dy = getPairwiseSeparations(r, pos)
	rho = np.sum(m * W(dx, dy, h), axis=1).reshape((M,1))
	return rho

L = 3000
ns = [int(i) for i in 3 ** np.arange(5, 5+4)]

max_rc = 0
fig_dens, ax_dens = plt.subplots(2, 2, figsize=(7, 5))
fig_repr, ax_repr = plt.subplots(2, 2, figsize=(7, 5), subplot_kw={'projection': '3d'})

# loc1 = np.random.uniform(-0.4, 0.4, 2)
# loc2 = np.random.uniform(-0.4, 0.4, 2)
# loc3 = np.random.uniform(-0.4, 0.4, 2)

RCs = []
ax_densf = ax_dens.flatten()
ax_reprf = ax_repr.flatten()
for i, n in enumerate(ns):
	_m = 1/L
	# positions1 = np.random.normal(loc=loc1, scale=0.1, size=(n//3, 2))
	# positions2 = np.random.normal(loc=loc2, scale=0.1, size=(n//3, 2))
	# positions3 = np.random.normal(loc=loc3, scale=0.1, size=(n//3, 2))
	# positions = np.concatenate((positions1, positions2, positions3), axis=0)
	positions = np.random.uniform(-0.5, 0.5, (n, 2))
	positions = positions[(positions[:, 0] >= -0.5) & (positions[:, 0] <= 0.5) & (positions[:, 1] >= -0.5) & (positions[:, 1] <= 0.5)]
	

	radii = np.random.uniform(0.01, 30, len(positions)) * _m
	areas = np.pi* (radii)**2
	print(f'Percentage of plant covered area: {np.sum(areas)/((L * _m)**2):.3f} % ')

	bw = 100*_m
	res = 100

	x = np.linspace(-0.5, 0.5, res)
	X, Y = np.meshgrid(x, x)
	XY_vstack = np.vstack([X.ravel(), Y.ravel()])
	
	# Define the grid on which to evaluate the KDE
	
	Z = getDensity(XY_vstack.T, positions, areas, bw).reshape(X.shape)
	lq = 12500e-5
	sg =  2500e-5
	RCs.append((Z + lq) * sg)
 
	ax_densf[i].scatter(positions[:, 0], positions[:, 1], s=areas*2000, c='white', alpha=0.5)
 
max_rc = np.max(RCs)
norm = plt.Normalize(vmin=0, vmax=max_rc)
levels = np.linspace(0, max_rc, 10)

for i, ax in enumerate(ax_densf):
	if i < len(ns):
		im = ax_densf[i].imshow(RCs[i], origin='lower', extent=(x[0], x[-1], x[0], x[-1]), cmap='plasma', norm=norm)
		ax.set_aspect('equal', adjustable='box')
		ax.set_title(f'n = {ns[i]}', fontsize = 7)
		ax.set_xticks([])
		ax.set_yticks([])

# Add a single colorbar for the entire figure
cbar = fig_dens.colorbar(im, ax=ax_densf, label='Reproduction Chance', fraction=0.046, pad=0.04, norm=norm)

for i, ax in enumerate(ax_repr.flatten()):
	if i < len(ns):
		ax.set_zlim(0, max_rc)
		contour = ax.contourf(X, Y, RCs[i], zdir='z', levels=levels, cmap='plasma', alpha=1)
		ax.set_proj_type('ortho')
		ax.set_title(f'n = {ns[i]}', fontsize = 7)
		ax.set_xticklabels([])
		ax.set_yticklabels([])
		ax.set_zticklabels([])
		ax.set_xlabel('')
		ax.set_ylabel('')
		ax.set_zlabel('')

# Add a single colorbar for the entire 3D figure
cbar_repr = fig_repr.colorbar(contour, ax=ax_reprf, label='Reproduction Chance', fraction=0.03, pad=0.1, norm=norm)
cbar_repr.ax.set_yticklabels([f'{(level*100):.1f} %' for level in levels], fontsize=6)

fig_dens.dpi = 100
fig_repr.dpi = 100
# fig_dens.tight_layout()
# fig_repr.tight_layout();
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from noise import pnoise2
import time
def generate_perlin_noise_2d(shape, res):
    def f(x, y):
        return pnoise2(x / res, y / res, octaves=4, persistence=0.5, lacunarity=2.0, repeatx=res, repeaty=res, base=0)
    
    return np.array([[f(i, j) for j in range(shape[1])] for i in range(shape[0])])

res = 100
values = generate_perlin_noise_2d((res, res), res/2)

dx = 1/res
xx = np.linspace(-0.5 + dx/2, 0.5 - dx/2, res)
X, Y = np.meshgrid(xx, xx)
XY_vstack = np.vstack([X.ravel(), Y.ravel()])

print(f'{X.shape = }, {Y.shape = }, {XY_vstack.shape = }')

pos = np.random.uniform(-0.5, 0.5, (100000, 2))


## KDTREE METHOD
start_time = time.time()
KDTree = KDTree(XY_vstack.T)
dist, idx = KDTree.query(pos)
idx_2d = np.unravel_index(idx, (res, res))
vals1 = values[idx_2d]
print(f'KDTree method took {time.time() - start_time:.3f} seconds')


# ## ARGMIN METHOD
# start_time = time.time()
# idx = np.argmin(np.sum((XY_vstack.T[:, None] - pos)**2, axis=2), axis=0)
# idx_2d = np.unravel_index(idx, (res, res))
# vals2 = values[idx_2d]
# print(f'Argmin method took {time.time() - start_time:.3f} seconds')


fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
# im = ax.imshow(values, origin='lower', extent=(-0.5, 0.5, -0.5, 0.5), cmap='viridis')
ax.scatter(pos[:, 0], pos[:, 1], c=vals1, cmap='viridis', s=5, alpha=1)
fig.colorbar(im, ax=ax)

# fig, ax = plt.subplots(1, 1, figsize=(8, 8))
# ax.set_aspect('equal', adjustable='box')
# ax.set_xlim(-0.5, 0.5)
# ax.set_ylim(-0.5, 0.5)
# im = ax.imshow(values, origin='lower', extent=(-0.5, 0.5, -0.5, 0.5), cmap='magma')
# ax.scatter(pos[:, 0], pos[:, 1], c=vals2, cmap='magma', s=5, alpha=1)
# fig.colorbar(im, ax=ax)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xx = np.linspace(-0.5, 0.5, 100)
X, Y = np.meshgrid(xx, xx)
grid_points = XY_vstack = np.vstack([X.ravel(), Y.ravel()]).T

vals = np.random.rand(len(grid_points))
print(vals.shape)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_aspect('equal', adjustable='box')
ax.scatter(grid_points[:, 0], grid_points[:, 1], c=vals, cmap='viridis', s=12, alpha=1, marker='s')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from mods.simulation import Simulation

L = 100  # m
half_width = half_height = 0.5
_m = 1/L
dispersal_range = 90 # m

num_plants = 10_000
n_iter = 10
lq = 0.01
sg = 0.1

dens0 = num_plants / L**2

plant_kwargs = {
    'r_min': 0.1 * _m,
    'r_max': 30 * _m,
    'growth_rate': 0.1 * _m,
    'dispersal_range': dispersal_range * _m,
    'species_germination_chance': sg,
    'is_dead': False,
    'is_colliding': False,
    'generation': 0,
}

buffer_size = 15
buffer_skip = 10
buffer_preset_times = np.linspace(0, n_iter, buffer_size).astype(int)

sim_kwargs = {
    'n_iter': n_iter,
    'half_width': half_width,
    'half_height': half_height,

    'dens0': dens0,
    '_m': _m,
    'num_plants': num_plants,
    'land_quality': lq,

    'kt_leafsize': 10,

    'density_check_radius': 100 * _m,
    'density_field_resolution': 100,

    'density_field_buffer_size': buffer_size,
    'density_field_buffer_skip': buffer_skip,
    'density_field_buffer_preset_times': buffer_preset_times,

    'state_buffer_size': buffer_size,
    'state_buffer_skip': buffer_skip,
    'state_buffer_preset_times': buffer_preset_times,
}

sim1 = Simulation(**sim_kwargs)
sim1.initiate_uniform_lifetimes(n = num_plants, t_min = 1, t_max = 300, **plant_kwargs)
fig11, ax11 = sim1.plot(title='Initial plant distribution', size = 5)
sim1.step()
fig12, ax12 = sim1.plot(title='Initial plant distribution', size = 5)

sim2 = Simulation(**sim_kwargs)
sim2.initiate_dense_distribution(n = num_plants, **plant_kwargs)
fig21, ax21 = sim2.plot(title='Initial plant distribution (dense forest)', size = 5)
sim2.step()
fig22, ax22 = sim2.plot(title='Initial plant distribution (dense forest)', size = 5)

plt.show()


In [ ]:
import numpy as np
from scipy.stats import gaussian_kde


class CustomKDE:
    def __init__(self, dataset, weights, bw_method):
        """
        Initialize the CustomKDE object.

        Parameters:
        -----------
        dataset : array_like
            The data points to estimate the density from.
        weights : array_like
            The weights (areas) of the data points.
        bw_method : scalar
        """
        self.dataset = np.atleast_2d(dataset)
        self.weights = np.atleast_1d(weights)
        self.bw_method = bw_method
        self.n, self.d = self.dataset.shape

        self.covariance = np.eye(self.d)

        # Compute the bandwidth matrix
        self.bandwidths = np.array([self.covariance * self.bw_method / weight for weight in self.weights])

    def evaluate(self, points):
        """
        Evaluate the estimated density at given points.

        Parameters:
        -----------
        points : array_like
            The points at which to evaluate the density.

        Returns:
        --------
        density : ndarray
            The estimated density values at the given points.
        """
        points = np.atleast_2d(points)
        densities = np.zeros(points.shape[1])

        for i, point in enumerate(points.T):
            # print(f'CustomKDE.evaluate: {i = }')
            # print(f'CustomKDE.evaluate: {self.dataset.shape = }, {point.shape = }')
            diff = self.dataset - point
            
            # print(f'CustomKDE.evaluate: {diff.shape = }, {self.bandwidths.shape = }')
            kernel_values = np.sum([np.exp(-0.5 * np.sum((diff @ np.linalg.inv(bandwidth)) * diff, axis=1)) for bandwidth in self.bandwidths])
            kernel_values = kernel_values.T
            
            # print(f'CustomKDE.evaluate: {kernel_values.shape = }, {self.weights.shape = }')
            
            densities[i] = np.sum(kernel_values)
                       

        return densities

    __call__ = evaluate


np.random.seed(0)
m2pp = 10000
ns = [10, 100, 1000, 10000]

for i, n in enumerate(ns[1:2]):
    fig, ax = plt.subplots(figsize=(4, 4))
    _m = float(np.sqrt(1/(m2pp * n)))
    # Define the grid on which to evaluate the KDE

    positions1 = np.random.normal(loc=[-0.3, -0.3], scale=0.1, size=(n//3, 2))
    positions2 = np.random.normal(loc=[0.3, 0.3], scale=0.1, size=(n//3, 2))
    positions3 = np.random.normal(loc=[0.0, 0.0], scale=0.1, size=(n//3, 2))
    positions = np.concatenate((positions1, positions2, positions3), axis=0)
    positions = positions[(positions[:, 0] >= -0.5) & (positions[:, 0] <= 0.5) & (positions[:, 1] >= -0.5) & (positions[:, 1] <= 0.5)]
    kt = KDTree(positions, leafsize=10)
    radii = np.random.uniform(0.01, 30, len(positions)) * _m
    areas = np.pi* radii**2

    bw = 1/100000
    res = 100

    x = np.linspace(-0.5, 0.5, res)
    X, Y = np.meshgrid(x, x)
    XY_vstack = np.vstack([X.ravel(), Y.ravel()])

    # Calculate the kernel density estimate
    kde = CustomKDE(positions, bw_method=bw, weights=radii)
    Z = kde(XY_vstack).reshape(X.shape)
    im = ax.imshow(Z, origin='lower', extent=(x[0], x[-1], x[0], x[-1]), cmap='Greys')
    # ax.contour(X, Y, Z, levels=[0.1, 0.5, 1.0], colors=['r', 'g', 'b'][::-1], alpha = 0.5, linewidths=1000*_m)
    ax.scatter(positions[:, 0], positions[:, 1], s=areas*1000, c='magenta', alpha=0.3)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f'n = {n}')
    fig.colorbar(im, ax=ax)

fig.dpi = 100
fig.tight_layout()

(100, 2)
Collision detection took 0.001 seconds
Density calculation took 0.157 seconds
Integrated density:	 0.0946372811207903
Sum of areas:		 0.09446420674500165


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from mpl_toolkits.mplot3d import Axes3D
import time


def boundary_check(pos, r):
    is_close_left_boundary = pos[0] - r < -0.5
    is_close_right_boundary = pos[0] + r > 0.5
    is_close_bottom_boundary = pos[1] - r < -0.5
    is_close_top_boundary = pos[1] + r > 0.5
    is_close_boundary = np.array(
        [is_close_left_boundary, is_close_right_boundary, is_close_bottom_boundary, is_close_top_boundary])
    return is_close_boundary


# Generate gridpoints and the KDTree
xx = np.linspace(-0.5, 0.5, 100)
X, Y = np.meshgrid(xx, xx)
grid_points = XY_vstack = np.vstack([X.ravel(), Y.ravel()]).T
kdtree_grid = KDTree(grid_points)

# Generate random points, radii and areas
N = 100
L = 1000
pos = np.random.uniform(-0.5, 0.5, (N, 2))
print(pos.shape)
kdtree_pos = KDTree(pos)
radii = np.random.uniform(1, 30, N)/L
radiiSq = radii**2
areas = np.pi * radiiSq

bandwidth_factor = 2
query_radius_factor = 3


### COLLISION DETECTION ###
start_time = time.time()
# Compute the sparse distance matrix
sparse_dist_matrix = kdtree_pos.sparse_distance_matrix(
    kdtree_pos, max_distance=2*np.max(radii))
# Filter pairs that are within collision distance
collision_dist_matrix = radii[:, None] + radii[None, :]
collisions = [(i, j) for i, j in zip(*sparse_dist_matrix.nonzero())
              if sparse_dist_matrix[i, j] < collision_dist_matrix[i, j]]
end_time = time.time()
print(f'Collision detection took {end_time - start_time:.3f} seconds')


### PLOT COLLISIONS ###
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.set_aspect('equal', adjustable='box')
for i, p in enumerate(pos):
    query_radius = query_radius_factor * \
        bandwidth_factor * radii[i]

    is_colliding = i in [c[0] for c in collisions]
    color = 'k'
    if is_colliding:
        color = 'purple'
    circle = plt.Circle(pos[i], radii[i], edgecolor=color, facecolor='none')
    ax.add_patch(circle)

    is_close_boundary = boundary_check(p, query_radius).astype(int)
    if is_close_boundary[0]:
        color = 'r'
    elif is_close_boundary[1]:
        color = 'g'
    elif is_close_boundary[2]:
        color = 'b'
    elif is_close_boundary[3]:
        color = 'y'
    circle = plt.Circle(pos[i], query_radius,
                        edgecolor=color, facecolor='none', ls='--', alpha=0.3)
    ax.add_patch(circle)
    if is_colliding:
        for j in [c[1] for c in collisions if c[0] == i]:
            ax.plot([pos[i, 0], pos[j, 0]], [
                    pos[i, 1], pos[j, 1]], color=color, alpha=1)

# Remove the point with the smallest radius for every pair of colliding points
collisions_set = set()
for i, j in collisions:
    if radii[i] < radii[j]:
        collisions_set.add(i)
    else:
        collisions_set.add(j)

# Filter out the colliding points
pos = np.array([p for k, p in enumerate(pos) if k not in collisions_set])
radii = np.array([r for k, r in enumerate(radii) if k not in collisions_set])
radiiSq = np.array([r for k, r in enumerate(
    radiiSq) if k not in collisions_set])
areas = np.array([a for k, a in enumerate(areas) if k not in collisions_set])


### DENSITY FIELD CALCULATION ###
start_time = time.time()
# Calculate the density field with periodic boundaries
density_field = np.zeros(grid_points.shape[0])
# for each plant, calculate its contribution to the density field based on its radius, it's area and the gaussian distribution
for i, p in enumerate(pos):
    sigmaSq = bandwidth_factor**2 * radiiSq[i]
    query_radius = query_radius_factor * \
        bandwidth_factor * radii[i]
    is_close_boundary = boundary_check(p, query_radius)

    # Apply periodic boundary conditions
    shifts = np.array([[1, 0], [-1, 0], [0, 1], [0, -1]])
    pos_shifted = [p + shift for shift in shifts[is_close_boundary]]

    # Calculate the density field
    for point in [p] + pos_shifted:
        for idx in kdtree_grid.query_ball_point(point, r=query_radius):
            distSq = np.sum((point - grid_points[idx])**2)
            density_field[idx] += areas[i] * \
                np.exp(-distSq / (2 * sigmaSq)) / (2 * np.pi * sigmaSq)


end_time = time.time()
print(f'Density calculation took {end_time - start_time:.3f} seconds')

# Integrate the density field to a single value accounting for the fact that it is 2-dimensional
integrated_density = np.sum(density_field) * (xx[1] - xx[0])**2
print(f'Integrated density:\t {integrated_density}')
print(f'Sum of areas:\t\t {sum(areas)}')

# Plotting the circles and the density field
# fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.set_aspect('equal', adjustable='box')
ax.set_facecolor(plt.cm.Greys(0))
for i, p in enumerate(pos):
    query_radius = query_radius_factor * \
        bandwidth_factor * radii[i]
    color = 'white'
    circle = plt.Circle(pos[i], radii[i], edgecolor=color, facecolor='none')
    ax.add_patch(circle)
im = ax.imshow(density_field.reshape(X.shape), origin='lower',
               extent=(-0.5, 0.5, -0.5, 0.5), cmap='Greys')


fig3d = plt.figure(figsize=(5, 5))
ax3d = fig3d.add_subplot(111, projection='3d')

X_flat = X.flatten()
Y_flat = Y.flatten()
Z_flat = density_field.flatten()

ax3d.plot_trisurf(X_flat, Y_flat, Z_flat, cmap='Greys',
                  linewidth=0.2, antialiased=True)
ax3d.set_xlabel('X')
ax3d.set_ylabel('Y')
ax3d.set_zlabel('Density')

plt.show()

# Parsing of simulation print statements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = """
| t =  100        |       P =  644        |       B = 0.05207     |
| t =  200        |       P =  748        |       B = 0.06267     |
| t =  300        |       P =  815        |       B = 0.07182     |
| t =  400        |       P =  808        |       B = 0.06849     |
| t =  500        |       P =  836        |       B = 0.07304     |
| t =  600        |       P =  807        |       B = 0.07063     |
| t =  700        |       P =  860        |       B = 0.07675     |
| t =  800        |       P =  879        |       B = 0.07456     |
| t =  900        |       P =  926        |       B = 0.07634     |
| t = 1000        |       P =  967        |       B = 0.08013     |
| t = 1100        |       P =  1007       |       B = 0.08518     |
| t = 1200        |       P =  1070       |       B = 0.08663     |
| t = 1300        |       P =  1100       |       B = 0.08679     |
| t = 1400        |       P =  1121       |       B = 0.09376     |
| t = 1500        |       P =  1153       |       B = 0.09278     |
| t = 1600        |       P =  1203       |       B = 0.09361     |
| t = 1700        |       P =  1246       |       B = 0.09777     |
| t = 1800        |       P =  1263       |       B = 0.10541     |
| t = 1900        |       P =  1328       |       B = 0.10359     |
| t = 2000        |       P =  1371       |       B = 0.10453     |
| t = 2100        |       P =  1431       |       B = 0.11115     |
| t = 2200        |       P =  1431       |       B = 0.11139     |
| t = 2300        |       P =  1382       |       B = 0.11549     |
| t = 2400        |       P =  1324       |       B = 0.10479     |
| t = 2500        |       P =  1318       |       B = 0.10712     |
| t = 2600        |       P =  1266       |       B = 0.10418     |
| t = 2700        |       P =  1286       |       B = 0.10384     |
| t = 2800        |       P =  1350       |       B = 0.10639     |
| t = 2900        |       P =  1392       |       B = 0.10853     |
| t = 3000        |       P =  1437       |       B = 0.11553     |
| t = 3100        |       P =  1432       |       B = 0.11837     |
| t = 3200        |       P =  1420       |       B = 0.11561     |
| t = 3300        |       P =  1440       |       B = 0.11654     |
| t = 3400        |       P =  1419       |       B = 0.11154     |
| t = 3500        |       P =  1468       |       B = 0.12068     |
| t = 3600        |       P =  1462       |       B = 0.11258     |
| t = 3700        |       P =  1446       |       B = 0.11453     |
| t = 3800        |       P =  1528       |       B = 0.11916     |
| t = 3900        |       P =  1592       |       B = 0.12116     |
| t = 4000        |       P =  1641       |       B = 0.12804     |
| t = 4100        |       P =  1649       |       B = 0.13371     |
| t = 4200        |       P =  1592       |       B = 0.12985     |
| t = 4300        |       P =  1521       |       B = 0.12119     |
| t = 4400        |       P =  1526       |       B = 0.12564     |
| t = 4500        |       P =  1489       |       B = 0.12393     |
| t = 4600        |       P =  1468       |       B = 0.11967     |
| t = 4700        |       P =  1457       |       B = 0.11847     |
| t = 4800        |       P =  1451       |       B = 0.11811     |
| t = 4900        |       P =  1518       |       B = 0.12357     |
| t = 5000        |       P =  1503       |       B = 0.11639     |
| t = 5100        |       P =  1535       |       B = 0.12437     |
| t = 5200        |       P =  1570       |       B = 0.12325     |
| t = 5300        |       P =  1533       |       B = 0.12497     |
| t = 5400        |       P =  1557       |       B = 0.13044     |
"""

# Parse the data
rows = data.strip().split('\n')
parsed_data = []

for row in rows:
    parts = row.split('|')
    t = int(parts[1].split('=')[1].strip())
    P = int(parts[2].split('=')[1].strip())
    B = float(parts[3].split('=')[1].strip())
    parsed_data.append([t, P, B])

parsed_data = np.array(parsed_data)

# Plot the data
fig, ax1 = plt.subplots(figsize=(10, 5))

red = (1, 0, 0, 0.3)
blue = (0, 0, 1, 0.3)

ax2 = ax1.twinx()
line1, = ax1.plot(parsed_data[:, 0], parsed_data[:, 1], label='Population', color=red, marker='o', lw=2, ms=5, markerfacecolor='white')
line2, = ax2.plot(parsed_data[:, 0], parsed_data[:, 2], label='Biomass', color=blue, marker='o', lw=2, ms=5, markerfacecolor='white')

red = (1, 0, 0, 1)
blue = (0, 0, 1, 1)

ax1.set_xlabel('Time [days]')
ax1.set_ylabel('Population', color=red)
ax2.set_ylabel('Biomass [kg]', color=blue)

ax1.tick_params(axis='y', labelcolor=red)
ax2.tick_params(axis='y', labelcolor=blue)

# Create a fused legend
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper left')

fig.suptitle('Parsed print statements', fontsize=16)
fig.tight_layout()
plt.show()
print(parsed_data[:, 0][-1])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
uni = np.random.uniform(0, 1, 100000)
norm = np.abs(np.random.normal(0.1, 1, 100000))

bins = np.linspace(0, 100, 1000)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
uni_integral = np.sum(uni * (bins[1] - bins[0])) / len(uni)
uni = uni/uni_integral
ax[0].hist(uni, bins=bins, alpha=0.5, color='b', label=f'Integral: {uni_integral:.3f}')
print('uni_integral = ', np.sum(uni * (bins[1] - bins[0])) / len(uni))
ax[0].set_xlim(bins[0], bins[-1])
ax[0].set_title('Uniform distribution')
ax[0].set_xlabel('Value')
ax[0].set_ylabel('Frequency')
ax[0].legend()

norm_integral = np.sum(norm * (bins[1] - bins[0])) / len(norm)
norm = norm/norm_integral
ax[1].hist(norm, bins=bins, alpha=0.5, color='r', label=f'Integral: {norm_integral:.3f}')
print('norm_integral = ', np.sum(norm * (bins[1] - bins[0])) / len(norm))
ax[1].set_xlim(bins[0], bins[-1])
ax[1].set_title('Normal distribution')
ax[1].set_xlabel('Value')
ax[1].set_ylabel('Frequency')
ax[1].legend()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Data
dispersal_range = np.array([55e-3, 50e-3, 45e-3, 40e-3, 35e-3, 30e-3, 25e-3, 20e-3, 15e-3, 10e-3]) * 3000/30 * 30
sim_len = np.array([7000, 3800, 3500, 4500, 4800, 4000, 2500, 1400, 2700, 900])

print(f'dispersal_range = {dispersal_range[::-1]}')
print(f'sim_len = {sim_len[::-1]}')

# Plot
plt.figure(figsize=(10, 3))
plt.plot(dispersal_range, sim_len, marker='o', linestyle='-', color='w')
plt.gcf().set_facecolor(np.array([30/255, 30/255, 30/255, 1]))
plt.gca().set_facecolor(np.array([30/255, 30/255, 30/255, 1]))
plt.tick_params(axis='x', colors='grey')
plt.tick_params(axis='y', colors='grey')
plt.xticks(dispersal_range)
plt.xlabel('Dispersal Range (m)', color='white')
plt.ylabel('Simulation Length', color='white')
plt.title('Dispersal Range vs Simulation Length (lq1e-1, sg25e-3, gaussian)', color='white')
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.grid(True, color = 'grey', linewidth = 0.5)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

density = np.array([2000, 1800, 1600, 1400, 1200, 1000, 800, 600, 400, 200]) * 1/(3000**2)
sim_len_density = np.array([5100, 3600, 3400, 6600, 4500, 8100, 5200, 3900, 4100, 3600])
 
# Plot
plt.figure(figsize=(10, 3))
plt.plot(density, sim_len_density, marker='o', linestyle='-', color='w')
plt.gcf().set_facecolor(np.array([30/255, 30/255, 30/255, 1]))
plt.gca().set_facecolor(np.array([30/255, 30/255, 30/255, 1]))
plt.tick_params(axis='x', colors='grey')
plt.tick_params(axis='y', colors='grey')
plt.xticks(density, [f'{d:.2e}' for d in density])
plt.xlabel('Density (plants/m$^2$)', color='white')
plt.ylabel('Simulation Length', color='white')
plt.title('Density vs Simulation Length (lq10000e-5, sg2500e-5, gaussian)', color='white')
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.grid(True, color = 'grey', linewidth = 0.5)
plt.show()

In [ ]:
precipitation = 1000
def precipitation_func(t): return precipitation

precipitation_func.__name__ = str(precipitation)

print(type(lambda: None))
print(type(precipitation_func))
print(precipitation_func.__name__)
print(isinstance(precipitation_func, type(lambda: None)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from numba import njit, prange
from scipy.interpolate import RegularGridInterpolator
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1 import make_axes_locatable

SQRT2 = np.sqrt(2)
PI = np.pi
SQRTPI = np.sqrt(PI)


@njit
def W(x, y, hSq):
    rSq = x**2 + y**2
    w = 1/(2 * PI * hSq) * np.exp(-rSq / (2 * hSq))
    return w


@njit
def getPairwiseSeparations(ri, rj):
    M = ri.shape[0]
    N = rj.shape[0]

    dx = np.empty((M, N))
    dy = np.empty((M, N))

    for i in prange(M):
        for j in prange(N):
            dx[i, j] = ri[i, 0] - rj[j, 0]
            dy[i, j] = ri[i, 1] - rj[j, 1]

    return dx, dy


@njit(parallel=True)
def getDensity(r, pos, m, hSq):
    M = r.shape[0]
    N = pos.shape[0]

    dx, dy = getPairwiseSeparations(r, pos)

    rho = np.zeros(M)
    for i in prange(M):
        for j in prange(N):
            rho[i] += m[j] * W(dx[i, j], dy[i, j], hSq)

    return rho.reshape((M, 1))

x = np.linspace(-0.5, 0.5, 100)
X, Y = np.meshgrid(x, x)

XY_vstack = np.vstack([X.ravel(), Y.ravel()])
XY_vstack = XY_vstack.T

positions = np.random.uniform(-0.5, 0.5, (39, 2))

dx, dy = getPairwiseSeparations(XY_vstack, positions)

densities = getDensity(XY_vstack, positions, np.ones(39), 0.005).reshape(X.shape)

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(densities, origin='lower', extent=(-0.5, 0.5, -0.5, 0.5), cmap='Greys_r')
# ax.quiver(XY_vstack[:, 0], XY_vstack[:, 1], dx.ravel(), dy.ravel())
ax.scatter(positions[:, 0], positions[:, 1], c='w', s=4)

def query(pos, xx, yy):
    i = int(np.argmin(np.abs(yy - pos[1])))
    j = int(np.argmin(np.abs(xx - pos[0])))    
    return densities[i, j]

grid = np.zeros(XY_vstack.shape[0])
for k, pos in enumerate(XY_vstack):
    grid[k] = query(pos, x, x)

grid = grid.reshape(X.shape)

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(grid, origin='lower', extent=(-0.5, 0.5, -0.5, 0.5), cmap='Greys_r')
ax.scatter(positions[:, 0], positions[:, 1], c='w', s=4)
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from mods.plant import Plant
from mods.fields import DensityFieldSPH, DensityFieldSimple
import json
import time

with open('../../default_kwargs.json', 'r') as file:
    kwargs = json.load(file)

L = 1000
num_plants = 1000
positions = np.random.uniform(-0.5, 0.5, (num_plants, 2))
radii = np.random.uniform(1, 5, num_plants) * 1/L
areas = np.pi * radii**2
state = [Plant(id = i , x = positions[i][0], y = positions[i][1], r=radii[i], **kwargs) for i in range(num_plants)]

density_check_radius = 90 * 1/L

resolution = 15
dfSPH = DensityFieldSPH(0.5, 0.5, density_check_radius, resolution)
dfSPH.update(state)    

resolution = 100
dfSPH = DensityFieldSPH(0.5, 0.5, density_check_radius, resolution)
dfSPH.update(state)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))
for i in range(10):
    target_population = 1000
    iterations = 1000
    n = 0
    k = 0.00
    N = []
    T = []
    color = 'red'
    for t in range(1, iterations):
        drive = n - target_population*1.05
        k_eff = t/iterations
        dn = - int(k_eff * drive)
        n += dn 
        if n > 10:
            n -= np.random.randint(0, int(n/10))
        
        print(f'{t = :<4.0f}   {n = :<4.0f}   {dn = :<5.0f}   {drive = :<5.0f}   {k_eff = :<4.4f}')
        N.append(n)
        T.append(t)
        if n > target_population:
            color = 'green'
            continue
    
    ax.plot(T, N, color=color, alpha=0.5)
plt.show()


In [ ]:
def get_max_depth(d, level=1):
    """
    Recursively finds the maximum depth of a nested dictionary.

    Args:
        d (dict): The dictionary to check.
        level (int, optional): The current level of depth. Defaults to 1.

    Returns:
        int: The maximum depth of the dictionary.
    """
    if not isinstance(d, dict) or not d:
        return level
    return max(get_max_depth(value, level + 1) for value in d.values())

# Example usage:
nested_dict = {
    'a': 1,
    'b': {
        'c': 2,
        'd': {
            'e': 3,
            'f': {
                'g': 4
            }
        }
    }
}

print(get_max_depth(nested_dict))  # Output: 4

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt



x = np.linspace(-0.5, 0.5, 1000)

_m = 1/3000

vals = np.linspace(10, 100, 3) * _m



# Given 95th percentile for both distributions
percentile_95 = 200 * _m

# Calculate the scale parameter γ for the Cauchy distribution and the standard deviation σ for the Gaussian distribution
gamma_analytical = percentile_95 / sp.stats.cauchy.ppf(0.95)
sigma_analytical = percentile_95 / sp.stats.norm.ppf(0.95)

print(f'Analytical ratio γ/σ: {(gamma_analytical / sigma_analytical):.4f}')


fig, ax = plt.subplots(1, 1, figsize=(10, 6))


μ = 0

σ = sigma_analytical
γ = gamma_analytical


# Calculate the Cauchy PDF and normalize it
cauchy_pdf = (1 / (np.pi * γ * (1 + x**2 / γ**2)))
cauchy_pdf /= cauchy_pdf.sum()
# Calculate the Gaussian PDF and normalize it
gauss_pdf = (1 / (np.sqrt(2 * np.pi) * σ)) * np.exp(-0.5 * (x**2 / σ**2))
gauss_pdf /= gauss_pdf.sum()


# Calculate the 95th percentile
p95_cauchy = np.tan(np.pi * (0.95 - 0.5)) * γ
# Calculate the 95th percentile for Gaussian
p95_gauss = sp.stats.norm.ppf(0.95, loc=0, scale=σ)

ax.plot(x, gauss_pdf, label=f'Gaussian (σ={(σ/_m):.2f} m)', color='blue', alpha=0.5)
ax.axvline(p95_gauss, color='blue', alpha=0.5, ls= '--', label=f'Gaussian 95th percentile: {p95_gauss/_m:.0f} m')

ax.plot(x, cauchy_pdf, label=f'Cauchy (γ={(γ/_m):.2f} m)', color='red', alpha=0.5)
ax.axvline(p95_cauchy, color='red', alpha=0.5, ls= '--', label=f'Cauchy 95th percentile: {p95_cauchy/_m:.0f} m')


x_ticks = np.array([-1500, -1000, -500, 0, 500, 1000, 1500]) * _m

ax.set_xticks(x_ticks)

ax.set_xticklabels([str(np.round(x_ticks / _m,)) for x_ticks in x_ticks])

ax.set_xlim(-0.5, 0.5)
ax.set_ylim(0, 0.01)

ax.grid(True)

ax.legend()

plt.show()

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

_m = 1/3000
x = np.linspace(1e-7/_m, 0.5/_m, 3000)


fig, ax = plt.subplots(1, 1, figsize=(10, 6))

μg = 10
σg = 90

gauss_pdf = (1 / (np.sqrt(2 * np.pi) * σg)) * np.exp(-0.5 * ((x - μg)**2 / σg**2))
gauss_pdf /= gauss_pdf.sum()
ax.plot(x, gauss_pdf, label=f'Gaussian   (μ, σ) = ({(μg):.0f} m, {(σg):.0f} m)', color='red', alpha=0.5)

μc = 10
σc = 90

cauchy_pdf = (1 / (np.pi * σc * (1 + ((x - μc) / σc)**2)))
cauchy_pdf /= cauchy_pdf.sum()
ax.plot(x, cauchy_pdf, label=f'Cauchy      (μ, σ) = ({(μc):.0f} m, {(σc):.0f} m)', color='green', alpha=0.5)

L = 90
σl = 1.5
lognormal_pdf = (1 / (x * σl * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((np.log(x/L))**2 / σl**2))
lognormal_pdf /= lognormal_pdf.sum()
ax.plot(x, lognormal_pdf, label=f'Lognormal (μ, σ) = ({(μl):.0f} m, {(σl):.0f} m)', color='blue', alpha=0.5)
# ax.set_xlim(-0.5, 0.5)
# ax.set_ylim(0, 0.005)

ax.grid(True)

ax.legend()

plt.show()

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
σ = 1.5
L = 90

# Calculate the PDFs of the normal and lognormal distributions
x = np.linspace(1e-10, 10000, 100000)
normal_pdf = 2 * sp.stats.norm.pdf(x, loc=0, scale=90)
lognormal_pdf = sp.stats.lognorm.pdf(x, s=σ, scale=L)


ax.plot(x, normal_pdf, label=f'2 * Normal, σ = 90 m', ls='-', color = 'k')
ax.plot(x, lognormal_pdf, label=f'Lognormal, {σ = :.2f}, {L = } m', ls='--', color = 'k')
    

# ax.set_xlim(-0.1, lognorm_samples.max())
ax.set_xlim(-1, 1000)
ax.set_xlabel('Distance (m)')
ax.set_ylabel('Probability Density')
ax.legend()
plt.show()

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
σ = 1.5
L = 90

N = 1000000

# # Plot the histogram of the lognormal samples
bins = np.linspace(0, 1000, 500)
# Sample points from a lognormal distribution
normal_samples_loc1 = np.random.normal(loc=0, scale=L, size=N//2)
normal_samples_loc2 = np.random.normal(loc=1000, scale=L, size=N//2)
uniform_samples = np.random.uniform(low=bins[0], high=bins[-1], size=N//8)
lognorm_samples_loc1 = np.random.lognormal(mean = 0, sigma=σ, size=N//2) * L
lognorm_samples_loc2 = np.random.lognormal(mean = 0, sigma=σ, size=N//2) * (-L) + 1000

normal_uniform_samples = np.concatenate((normal_samples_loc1, normal_samples_loc2, uniform_samples))
lognorm_samples = np.concatenate((lognorm_samples_loc1, lognorm_samples_loc2))


freq1, bins = np.histogram(
    normal_uniform_samples, bins=bins, density=True)
freq2, bins = np.histogram(lognorm_samples, bins=bins, density=True)
midpoints = (bins[1:] + bins[:-1]) / 2
ax.hist(midpoints, bins=bins, weights=freq1, alpha=0.5, label='Normal + Uniform', histtype='stepfilled')
ax.hist(midpoints, bins=bins, weights=freq2, alpha=0.5, label='Lognormal', histtype='stepfilled')

ax.legend()

# Diffusion
$$\frac{\partial \phi}{\partial t} = D \frac{\partial^2 \phi}{\partial x^2}$$
where
- $\phi$ is the concentration in dimensions of $[ \mathrm N \mathrm L^{−3}]$, example $\mathrm{mol}/\mathrm{m}^3$; $\phi = \phi(x,t)$ is a function that depends on location $x$ and time $t$,
- $t$ is time, example $\mathrm s$,
- $D$ is the diffusion coefficient in dimensions of $[ \mathrm L^2  \mathrm T^{−1}]$, example $\mathrm{m}^2/\mathrm{s}$,
- $x$ is the position, example $\mathrm m$.

# Cutting of the data buffer

In [ ]:
import numpy as np
N = np.full((10, 3), np.nan)
N[:5] = np.arange(15).reshape(5, 3)

N[0, 2] = np.nan

values = N

print(values)

values = values[:np.where(~np.isnan(values).all(axis=1))[0][-1] + 1]

print()
print(values)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

c = 10
g_max = 0.05
k1 = 5
D_p = 0.1
α = 0.2
k2 = 5
W0 = 0.2
r_w = 0.2
D_w = 0.1
D_o = 100

d = 0.1
R = 0.75


def plot_fields(P, W, O, t):
    P_show = P.copy()
    W_show = W.copy()
    O_show = O.copy()

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f'Time: {t:.3f}', fontsize=16)
    im0 = ax[0].imshow(P_show, cmap='Greens')
    ax[0].set_title('P')
    fig.colorbar(im0, ax=ax[0])

    im1 = ax[1].imshow(W_show, cmap='Blues')
    ax[1].set_title('W')
    fig.colorbar(im1, ax=ax[1])

    im2 = ax[2].imshow(O_show, cmap='Blues')
    ax[2].set_title('O')
    fig.colorbar(im2, ax=ax[2])
    
    for a in ax:
        a.axis('off')


# Define the Laplacian operator for 2D fields
def laplacian(Z, dx):
    # add ghost nodes with copies of the values at the boundary
    Z = np.pad(Z, 1, mode='edge')
    
    Ztop = np.roll(Z, 1, axis=0)
    Zleft = np.roll(Z, 1, axis=1)
    Zbottom = np.roll(Z, -1, axis=0)
    Zright = np.roll(Z, -1, axis=1)
    lap = (Ztop + Zleft + Zbottom + Zright - 4 * Z) / dx**2
    return lap[1:-1, 1:-1]
    # return lap


# Initialize the fields
res = 100
N_gridpoints = int(res**2)
n = N_gridpoints // 100
coords = np.random.randint(0, res, (n, 2))

P = np.zeros((res, res))
P[coords[:, 0], coords[:, 1]] = -1
W = np.zeros((res, res)) + R/r_w
O = np.zeros((res, res)) + R/(α * W0)

plot_fields(P, W, O, t=0)
plt.show()

# Time step and spatial step
dt = 0.01
L = 400
dx = L/res

# Simulation loop
prev_mod = 1
for t in np.arange(1, 301, dt):
    print(f'{t = :.0f}', end='\r')
    ΔP = laplacian(P, dx)
    ΔW = laplacian(W, dx)
    ΔO = laplacian(O, dx)

    dPdt = c * g_max * W / (W + k1) * P - d * P + D_p * ΔP
    P += dPdt * dt
    dWdt = α * O * (P + k2 * W0) / (P + k2) - g_max * \
        W / (W + k1) * P - r_w * W + D_w * ΔW
    W += dWdt * dt
    dOdt = R - α * O * (P + k2 * W0) / (P + k2) + D_o * ΔO
    O += dOdt * dt

    mod = t % 100
    if mod < prev_mod:
        plot_fields(P, W, O, t)
        plt.show()
    prev_mod = mod

In [ ]:
import numpy as np
for i in range(10):
    n = 3.5    
    decimal_part = n - int(n)
    n = int(n)
    if np.random.rand() < decimal_part:
        n += 1
    
    print(n)

In [ ]:
import pandas as pd

real_df = pd.read_csv(
    '../../Data/temp/memory_test/size_buffer_bs8.csv', header=0)
test_df = pd.read_csv('../../test.csv', header=None)
print('bins' in real_df.iloc[0,:3].values)
print(real_df.iloc[0, :3])
print()
print('bins' in test_df.iloc[0,:].values)
print(test_df.iloc[0, :])

In [ ]:
import uuid
for _ in range(10):
    Nmax = int(1e6)
    ids = [uuid.uuid4().int for _ in range(Nmax)]
    n_duplicates = len(ids) - len(set(ids))
    print(f"Has {n_duplicates} duplicates")

In [ ]:
import numpy as np
import pandas as pd

# Load the DataFrame
path = '../../Data/temp/convergence_test/state_buffer_807225414.csv'
df = pd.read_csv(path, header=0)

# Get unique IDs and create a mapping
ids = df['id'].unique()
id_map = {old_id: new_id for new_id, old_id in enumerate(ids)}

# Replace IDs using the map function
df['id'] = df['id'].map(id_map)

# Save the modified DataFrame to a new CSV file
df.to_csv(path.replace('.csv', '_simple.csv'), index=False)

In [ ]:
import json

with open(f'../../default_kwargs.json', 'r') as file:
    kwargs = json.load(file)
print(kwargs)

In [ ]:
surfix = '247088000'

for i in range(100):
    surfix = f'{surfix.split("_")[0]}_{i+1}'
    print(surfix)


In [ ]:
import pandas as pd
from mods.buffers import DataBuffer

file_path = 'C:/Users/carla/Dropbox/_CARL/UNI/KANDIDAT/PROJEKT/Code/Data/Data/temp/memory_test/data_buffer_bs1_copy.csv'

dbuffer = DataBuffer(file_path)
print(dbuffer.get_data())

# new_data = pd.DataFrame({
#     'Time': [2001.0],
#     'Population': [1000],
#     'Biomass': [0.1],
# })

# data = pd.concat([data, new_data], ignore_index=True)
# print(data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for T in range(1, 10):
    n_cols = int(np.ceil(np.sqrt(T)))
    n_rows = int(np.ceil(T / n_cols))
    size = 2
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(2*n_cols, 2*n_rows))
    if n_rows == 1 and n_cols == 1:
        ax = np.array([ax])
    for i, a in enumerate(ax.flatten()):
        if i < T:
            a.text(0.5, 0.5, str(i+1), fontsize=12, ha='center')
            a.set_aspect('equal', adjustable='box')
            a.set_xticks([])
            a.set_yticks([])
            a.set_xticklabels([])
            a.set_yticklabels([])
        else:
            a.axis('off')

In [ ]:
import numpy as np
l = 10
n = 20

a = np.linspace(0, l, 5*l+1, endpoint=True)
b = np.linspace(a[0], a[-1], n, endpoint=True)

# Using numpy's searchsorted method for a more efficient approach
indices = np.searchsorted(a, b)
indices = np.clip(indices, 1, len(a) - 1)
left = a[indices - 1]
right = a[indices]
c = np.where(np.abs(b - left) < np.abs(b - right), left, right)

print(len(c))
print(c)

In [ ]:
import numpy as np
n_plots = np.inf
times_unique = np.array([17999., 21999., 26999., 33999., 37999., 39999., 43999., 44999.,
       46999., 51999.])
b = np.linspace(times_unique.min(), times_unique.max(), min(len(times_unique), n_plots))
times = np.unique([times_unique[np.abs(times_unique - t).argmin()] for t in b])
print(times)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
vals = np.random.normal(0, 1, (3, 1000))
bins = 25
histograms_list = []
bins_list = []
for row in vals:
    hist, bins = np.histogram(row, bins=bins)
    histograms_list.append(hist)
    bins_list.append(bins)


fig = plt.figure(figsize=(8, 5))
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])
if not isinstance(ax, np.ndarray):
    ax = np.array([ax])

y_min = min(hist.min() for hist in histograms_list)
y_max = max(hist.max() for hist in histograms_list)
x_min = min(bins.min() for bins in bins_list)
x_max = max(bins.max() for bins in bins_list)
for a in ax:
    a.set_xlim(x_min, x_max)
    a.set_ylim(y_min, y_max)

for i, (hist, bins) in enumerate(zip(histograms_list, bins_list)):
    ax[i].bar(bins[:-1], hist, width=np.diff(bins), align='edge', alpha=0.5)
plt.show()